In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from selenium.webdriver import ActionChains

import pandas as pd
import bs4
import time
import requests
from bs4 import BeautifulSoup
import re
import csv

options = webdriver.ChromeOptions()
driver_loc = "../externlib/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(driver_loc , options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정

driver.get("https://map.naver.com/v5/") #네이버 신 지도 
try:
    element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
    pass

name = "none"
menu = "none"
menuList=[]



search_box = driver.find_element_by_class_name("input_search")
search_box.send_keys("강남구 신사동 음식점")
search_box.send_keys(Keys.ENTER) #검색창에 "서울 칵테일바" 입력

time.sleep(3) #화면 표시 기다리기

element = driver.find_element_by_id("searchIframe")

driver.switch_to.frame(element) #프레임 이동
time.sleep(4) #화면 표시 기다리기



scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
#검색 결과로 나타나는 scroll-bar 포함한 div 잡고
driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
#여기까지 scroll
#맨 아래까지 내려서 해당 페이지의 내용이 다 표시되게 함



selectWeb = driver.find_elements_by_class_name("O9Z-o")
# 50개 선택창 아이프레임 들어갔다 나오기

flag = False
k = 0
while 1==1:
    
    for i in range(0, len(selectWeb)):
    # try:
        time.sleep(3)
        # 이쪽 페이지 있는거 포문으로 돌기
        ##검색된 거 하나하나 들어가는 곳
        selectWeb[i].click()
        time.sleep(3)

        driver.switch_to.default_content()  #프레임 빠져나오기

        element = driver.find_element_by_id("entryIframe")

        driver.switch_to.frame(element) #프레임 이동
        # 개별정보 접근시 프레임 이동
        time.sleep(2)
        ##현재 페이지 소스 가져오기
        html = driver.page_source
        time.sleep(2)
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")
        time.sleep(2)
        name = bs.find(attrs={"class":"_3XamX"}).text
        menuPrepare = bs.findAll(attrs={"class":"place_bluelink _1UOrB"})

        if not menuPrepare:
        ##
        # 메뉴 수집
            tempList = []
            menuName = bs.findAll(attrs={"class":"_1hkqi"})

            menuName[0].find(attrs={"class":"_1q3GD"}).text
            menuName[0].find(attrs={"class":"_3IAAc"}).text
            # ---------------------
            menusNamelist = ""

            menuCount = min(len(menuName),4)

            for j in range(0, len(menuName)):
                if j==0:
                    menusNamelist = ""
                    menusNamelist = menusNamelist + menuName[j].find(attrs={"class":"_1q3GD"}).text + "_" + menuName[j].find(attrs={"class":"_3IAAc"}).text
                else:
                    menusNamelist = menusNamelist + "|" + menuName[j].find(attrs={"class":"_1q3GD"}).text + "_" + menuName[j].find(attrs={"class":"_3IAAc"}).text
                

            tempList = [name, menusNamelist]

            menuList.append(tempList)
            menu = "none"
            name = "none"
        ##

        
        else:



            menuPrice = bs.findAll(attrs={"class":"_3zdzT"})

            menuCount = min(len(menuPrepare),4)

            tempList = [] 

            for a in range(0, menuCount):
                if a==0:
                    menu = ""
                    menu = menu + menuPrepare[a].text + "_" + menuPrice[a].text
                else:
                    menu = menu + "|" + menuPrepare[a].text +"_"+menuPrice[a].text

            tempList = [name, menu]

            menuList.append(tempList)

            menu = "none"
            name = "none"

    
    # except Exception as e:
    #     print(e)
    #     time.sleep(2)


        driver.switch_to.default_content()  #프레임 빠져나오기

        element = driver.find_element_by_id("searchIframe")
        driver.switch_to.frame(element) #프레임 이동
        # print(str(i)+"번쨰 완료")
        
        if k >=1:

            if menuList[-1] == menuList[-2]:
                menuList.remove(menuList[-1])
                flag = True
                break
            
    if flag:
        driver.switch_to.default_content()  #프레임 빠져나오기
        break
    ##다음 페이지로 넘기는 로직
    try:

        driver.switch_to.default_content()  #프레임 빠져나오기

        driver.switch_to.frame(element) #프레임 이동
        

        ##다음 페이지 이동
        nextPage= driver.find_elements_by_class_name("_3Dl4U")
        
        nextPage[1].click()
        time.sleep(2)


        k=k+1
        
        scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
        #검색 결과로 나타나는 scroll-bar 포함한 div 잡고
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
        time.sleep(2)
        #여기까지 scroll
        #맨 아래까지 내려서 해당 페이지의 내용이 다 표시되게 함
        
        selectWeb = driver.find_elements_by_class_name("O9Z-o")
        # 50개 선택창 아이프레임 들어갔다 나오기
        time.sleep(2)

    except:
        print("error")






C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1796/140811167.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_loc , options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1796/140811167.py:34: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  search_box = driver.find_element_by_class_name("input_search")
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1796/140811167.py:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("searchIframe")
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_1796/140811167.py:47: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  scroll_div = driver.find_element_by_xpat

In [2]:
menuList

[['꽁티드툴레아', '커피_5,500원|Latte_7,000원|Affogato_7,500원|아보카도토스트_14,000원'],
 ['에뚜왈', '마들렌_2,200원|크루와상_3,400원|뺑 오 쇼콜라_3,600원|퀸 아망_3,500원'],
 ['Just Steak',
  'T-Bone Steak (100g)_19,300원|L-Bone steak (100g)_15,300원|그릴에 구운 야채_12,500원|볼로네제_22,000원'],
 ['Bowl Room', '보울룸 시그니쳐_12,000원|쉬림프타코_4,000원|피쉬타코_4,000원|또띠아 칩_1,500원'],
 ['플르서라구뜨', '오믈렛_18,000원|오일파스타_20,000원|토마토스파게티_21,000원|크림스파게티_20,000원'],
 ['스시사카우', '런치 스시오마카세_60,000원|디너 스시오마카세_99,000원']]